In [1]:
from tpm.data_model import *
from tpm.util.io import read_geolife
from tpm.util.dist import haversine_distance
from tpm.preprocessing import time_duplication_filter
from tpm.preprocessing import speed_filter_abs
import numpy as np
import pandas as pd
import folium
from datetime import timedelta

In [2]:
trajs = read_geolife('/mnt/hdd1/christian/data/geotracking/Geolife Trajectories 1.3/Data/001/Trajectory')


In [3]:
preprocessed = list()
for traj in trajs:
    traj_new = time_duplication_filter(traj)
    traj_new = speed_filter_abs(traj_new, 300, in_kmh=True)
    preprocessed.append(traj_new)


In [4]:
def staypoints_geolife(traj):
    time_thresh = 30*60
    dist_thresh = 200

    staypoints = list()
    i, i_max = 0, len(traj)
    while i < i_max:
        j = i+1
        token = 0
        while j < i_max:
            dist = haversine_distance(traj[i], traj[j])
            if dist > dist_thresh:
                delta_time = traj[j].datetime - traj[i].datetime
                if delta_time.total_seconds() > time_thresh:
                    mean_point = np.mean([[p.lat, p.lon] for p in traj[i:j+1]], axis=0)
                    arrival_time = traj[i].datetime
                    leave_time = traj[j].datetime
                    staypoints.append([mean_point, arrival_time, leave_time, i, j])
                    i = j
                    token = 1
                break
            j = j+1
        if not token == 1:
            i = i+1

    
    return staypoints

In [5]:
len(trajs)

71

In [6]:
def make_df(trajs):
    data = list()
    for traj in trajs:
        fp = traj[0]
        sps = staypoints_geolife(traj)
        lp = traj[-1]               
        
        if len(sps) > 1:
            data.append([fp.lat, fp.lon, fp.datetime, sps[0][0][0], sps[0][0][1], sps[0][1]])
            for i in range(1, len(sps)-1):
                data.append([sps[i][0][0], sps[i][0][1], sps[i][1], sps[i+1][0][0], sps[i+1][0][1], sps[i+1][2]])
            data.append([sps[-1][0][0], sps[-1][0][1], sps[-1][2], lp.lat, lp.lon, lp.datetime])
        else:
            data.append([fp.lat, fp.lon, fp.datetime, lp.lat, lp.lon, lp.datetime])
        
        
    df = pd.DataFrame(data, columns=['start_lat','start_lon','start_date','end_lat','end_lon','end_date'])
    df = df.set_index(pd.DatetimeIndex(df['start_date'])).sort_index()
    return df

In [7]:
df = make_df(preprocessed)



,start_lat,start_lon,start_date,end_lat,end_lon,end_date
2008-10-23 14:53:05,39.984093,116.319237,2008-10-23 14:53:05,39.978466,116.327766,2008-10-23 15:00:33
2008-10-23 20:49:08,40.014435,116.305862,2008-10-23 20:49:08,40.013802,116.306534,2008-10-23 21:04:28
2008-10-24 08:41:04,40.013866,116.306473,2008-10-24 08:41:04,39.978931,116.326279,2008-10-24 09:11:29
2008-10-24 10:56:07,39.981266,116.309845,2008-10-24 10:56:07,39.981495,116.309296,2008-10-24 12:18:23
2008-10-24 11:28:19,39.981495,116.309296,2008-10-24 11:28:19,39.978592,116.326065,2008-10-24 14:30:46
2008-10-24 12:55:26,39.978592,116.326065,2008-10-24 12:55:26,39.981445,116.310600,2008-10-24 15:25:50
2008-10-24 15:25:50,39.981445,116.310600,2008-10-24 15:25:50,39.977898,116.327065,2008-10-24 15:35:50
2008-10-25 08:44:05,40.013813,116.306480,2008-10-25 08:44:05,39.993111,116.145844,2008-10-25 14:50:14
2008-10-25 17:24:19,39.989532,116.186752,2008-10-25 17:24:19,39.990788,116.193352,2008-10-25 19:07:22
2008-10-25 19:07:22,39.990788,116.193352,2008-10-25 19:07:22,40.013817,116.306480,2008-10-25 20:30:01


In [56]:
def cluster_into_spots(df, init_eps=300, levels=0, threshold=0.1):
    start_points = list()
    end_points = list()
    length = len(df)

    for i in range(length):
        start_points.append([df['start_lat'].iloc[i], df['start_lon'].iloc[i]])
        end_points.append([df['end_lat'].iloc[i], df['end_lon'].iloc[i]])

    points = np.radians(np.vstack([start_points, end_points]))

    haversine = DistanceMetric.get_metric('haversine')
    dist = haversine.pairwise(points) * R

    clusters = dbscan(dist, metric='precomputed', min_samples=1, eps=init_eps)[1]
    clusters = np.array(clusters, dtype=np.object)

    for _ in range(levels):
        init_eps = init_eps * 0.2
        counts = dict(Counter(clusters))
        for key in counts:
            if counts[key] > threshold * length:
                idxs = np.where(clusters == key)[0]
                dist = haversine.pairwise(points[idxs]) * R
                inner_clusters = dbscan(dist, metric='precomputed', min_samples=1, eps=init_eps)[1]
                for i, idx in enumerate(idxs):
                    clusters[idx] = "{}_{}".format(clusters[idx], inner_clusters[i])

    start_clusters = list()
    end_clusters = list()
    for i, cluster in enumerate(clusters):
        if i < length:
            start_clusters.append(clusters[i])
        else:
            end_clusters.append(clusters[i % length + length])

    df['start_cluster'] = start_clusters
    df['end_cluster'] = end_clusters
    return df

In [58]:
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import dbscan
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.neighbors import DistanceMetric

from tpm.util.dist import haversine_distance
from tpm.data_model import R

from collections import Counter

import numpy as np
import pandas as pd
df = cluster_into_spots(df)

In [162]:
def get_test_trajs(test, trajs):
    test_trajs = list()
    for i, row in test.iterrows():
        start = row['start_date']
        end = row['end_date']
        for traj in trajs:
            start_traj = None
            end_traj = None
            for i, p in enumerate(traj):
                if p.datetime == start:
                    start_traj = i
                    
                if p.datetime == end:
                    end_traj = i+1
            
            if start_traj is not None and end_traj is not None:
                test_traj = traj[start_traj:end_traj]
        
        test_trajs.append(test_traj)
        
    return test_trajs

In [163]:
test_trajs = get_test_trajs(df.iloc[81:], preprocessed)
[len(traj) for traj in test_trajs]

[697,
 863,
 721,
 491,
 482,
 516,
 538,
 505,
 145,
 566,
 858,
 495,
 766,
 470,
 692,
 59,
 33,
 117,
 780,
 648,
 534,
 603,
 771,
 1467,
 622,
 1961,
 852,
 835,
 244,
 352,
 888,
 919,
 1842,
 691,
 1395,
 5022,
 897,
 539]

In [137]:
from math import *
def calc_heading(a, b):
    lat1 = radians(a.lat)
    lon1 = radians(a.lon)
    lat2 = radians(b.lat)
    lon2 = radians(b.lon)

    bearing = atan2(cos(lat1) * sin(lat2) - sin(lat1) * cos(lat2) * cos(lon2 - lon1), sin(lon2 - lon1) * cos(lat2))
    bearing = degrees(bearing)
    bearing = (bearing + 360) % 360

    return bearing

In [147]:
first100points = test_trajs[0][:100]
headings = list()
for p1,p2 in zip(first100points[:-1], first100points[1:]):
    headings.append(calc_heading(p1,p2))

print(headings)

[81.2912472959996, 139.75971072243487, 112.67304485262332, 146.86191728331977, 179.9999558498423, 179.99995830320643, 190.8691096731516, 233.89236801448595, 266.1897866514836, 276.4733851698767, 304.57854807620333, 341.9224192381115, 326.8620934205903, 0.0, 0.0, 0.0, 326.86209482090817, 297.04856865058645, 9.87564645306287, 8.254609731795256, 4.148855635541906, 335.82834821017923, 294.99469299236836, 305.0714799433904, 338.11256474147353, 333.91250443457983, 326.86221585269107, 313.10946440895526, 308.7423936410574, 315.6008111553737, 323.2737365614159, 316.3445593822777, 318.9624762292708, 344.63045370485503, 9.875544728698117, 1.967919709355158, 1.9679198176206683, 10.278433606165493, 4.414946670294739e-05, 4.14881288383026, 4.665241896029386, 12.975121280063718, 1.9679215793815956, 5.885209877253999, 5.885210855518949, 7.82561992269973, 3.735184628986417, 6.571839670679481, 9.74814284857456, 357.80086492897055, 12.870636564406766, 3.9311707332121273, 4.148821898354129, 354.114874503

In [35]:
import numpy as np

In [38]:
np.median(headings)


270.0

In [42]:
from tpm.util.visualization import visualize_start_end_trajectory
from tpm.util.visualization import visualize_trajectory

In [88]:
visualize_trajectory(Trajectory(test_trajs[0][0:300]))

In [90]:
from masterthesis.util import add_dist
from masterthesis.models import FuzzyLocationMixin

In [295]:
class HeadingEstimator(FuzzyLocationMixin):
    def __init__(self, angle_threshold=60):
        self.angle_threshold = angle_threshold
    
    def fit(self, X, y=None):
        self.data_ = [(lat, lon, end) for lat, lon, end in
                      zip(X['end_lat'], X['end_lon'], X['end_cluster'])]

        self.data_ = pd.DataFrame(data=self.data_,
                                  columns=['end_lat', 'end_lon', 'end_cluster'])
        return self
    
    
    def predict(self, traj):
        start_p = traj[0]
        end_p = traj[-1]
        heading1 = calc_heading(start_p, end_p)
        heading_diffs = list()
        
        for i, row in self.data_.iterrows():
            p = Point(row['end_lat'], row['end_lon'], None)
            heading2 = calc_heading(start_p, p)
            heading3 = calc_heading(end_p, p)
            
            heading_diff1 = max(heading1, heading2) - min(heading1, heading2)
            heading_diff1 = self._correct_heading_diff(heading_diff1)
            
            heading_diff2 = max(heading1, heading3) - min(heading1, heading3)
            heading_diff2 = self._correct_heading_diff(heading_diff2)
            
            #print(heading_diff2)
            if heading_diff2 > self.angle_threshold:
                heading_diffs.append(180)
            else:
                heading_diffs.append(heading_diff1)
        

        
        return self.data_['end_cluster'].iloc[np.argmin(heading_diffs)]    
             
    
    def _correct_heading_diff(self, diff):
        if diff >=180:
                diff = 360 - diff
        return diff
    

In [296]:
he = HeadingEstimator()
he = he.fit(df[:81])

res = dict()
for traj in test_trajs:
    
    if len(traj) < 500:
        continue
    
    he.predict_proba(traj[0:400])
    res[traj[-1]] = he.predict_proba(traj[0:400])

[10.601133162129997, 11.7659459472859, 11.768787413176483, 11.774341222871158, 11.782536088337679, 11.785396224539227, 11.796189314349931, 11.799155222236095, 11.812752229185264, 11.826474762623093, 11.826514784243159, 11.840143279295035, 11.842927865564945, 11.842956210376997, 11.851198754005736, 11.853929433632686, 11.87042874485303, 11.873187080723255, 11.878684614891824, 11.878689026468066, 11.884180569579144, 11.91443563569328, 11.932615534554202, 45.34716708467187, 45.49080547553075, 45.72580669423985, 45.902644562957335, 45.947022359895755, 46.30679940987011, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180]
[13 44 18 39 50 67 57 62 48 75  9 33 78 35 25 60 15  1 51 80 69 21 31 65 66
 28 10  8  7 77 76 74 52 53 54 55 56 64 58 68 59 72 61 71 63 70 49 73  0 46
  2  3  4  5  6

In [297]:
ecs = {Point(lat, lon, None) : end for lat, lon, end in zip(df['end_lat'], df['end_lon'], df['end_cluster'])}
pcs = list()
eclus = list()
for re in res:
    pc = res[re]
    min_dist = 300
    eclu = None
    for ecp in ecs:
        dist = haversine_distance(re, ecp)
        if dist < min_dist:
            min_dist = dist
            eclu = ecs[ecp]
            
    pcs.append(pc)
    eclus.append(eclu)

Counter(np.array(pcs) == np.array(eclus))

Counter({False: 17, True: 11})

In [328]:
class HeadingMostFrequentTargetEstimator(FuzzyLocationMixin):
    def __init__(self, angle=25, angle_threshold=30):
        self.angle = angle
        self.angle_threshold = angle_threshold
    
    def fit(self, X, y=None):
        self.data_ = [(lat, lon, end) for lat, lon, end in
                      zip(X['end_lat'], X['end_lon'], X['end_cluster'])]

        self.data_ = pd.DataFrame(data=self.data_,
                                  columns=['end_lat', 'end_lon', 'end_cluster'])
        return self
    
    
    def predict(self, traj):
        try:
            return self.predict_proba(traj).index[0]
        except:
            return None
    
    def predict_proba(self, traj):
        length = len(self.data_)
        start_p = traj[0]
        end_p = traj[-1]
        heading1 = calc_heading(start_p, end_p)
        
        potential_clusters = list()
        heading_diffs = list()
        
        for i, row in self.data_.iterrows():
            p = Point(row['end_lat'], row['end_lon'], None)
            heading2 = calc_heading(start_p, p)
            heading3 = calc_heading(end_p, p)
            
            heading_diff1 = max(heading1, heading2) - min(heading1, heading2)
            heading_diff1 = self._correct_heading_diff(heading_diff1)
            
            heading_diff2 = max(heading1, heading3) - min(heading1, heading3)
            heading_diff2 = self._correct_heading_diff(heading_diff2)
            
            #print(heading_diff2)
            if heading_diff2 < self.angle_threshold and heading_diff1 < self.angle:
                potential_clusters.append(row['end_cluster'])
                heading_diffs.append(heading_diff1)
                
                
                
        args = np.argsort(heading_diffs)
        sorted_clusters = list()
        for arg in args:
            if not potential_clusters[arg] in sorted_clusters:
                sorted_clusters.append(potential_clusters[arg])
        
        
        return self.data_['end_cluster'].loc[self.data_['end_cluster'].isin(potential_clusters)].value_counts()/length
        
        
    def _correct_heading_diff(self, diff):
        if diff >=180:
                diff = 360 - diff
        return diff

    

In [329]:
he = HeadingMostFrequentTargetEstimator()
he = he.fit(df[:81])

res = dict()
for traj in test_trajs:
    
    if len(traj) < 600:
        continue
    print(he.predict_proba(traj[0:100]))
    res[traj[-1]] = he.predict(traj[0:100])

1     0.283951
16    0.037037
22    0.012346
12    0.012346
8     0.012346
Name: end_cluster, dtype: float64
15    0.049383
Name: end_cluster, dtype: float64
1     0.283951
15    0.049383
16    0.037037
22    0.012346
12    0.012346
8     0.012346
Name: end_cluster, dtype: float64
16    0.037037
6     0.012346
24    0.012346
Name: end_cluster, dtype: float64
3     0.419753
16    0.037037
25    0.012346
24    0.012346
22    0.012346
10    0.012346
8     0.012346
6     0.012346
Name: end_cluster, dtype: float64
3     0.419753
1     0.283951
15    0.049383
16    0.037037
9     0.037037
2     0.037037
25    0.012346
24    0.012346
23    0.012346
22    0.012346
12    0.012346
8     0.012346
Name: end_cluster, dtype: float64
2     0.037037
5     0.012346
21    0.012346
Name: end_cluster, dtype: float64
10    0.012346
25    0.012346
Name: end_cluster, dtype: float64
3     0.419753
15    0.049383
25    0.012346
Name: end_cluster, dtype: float64
Series([], Name: end_cluster, dtype: float64)
13 

In [330]:
ecs = {Point(lat, lon, None) : end for lat, lon, end in zip(df['end_lat'], df['end_lon'], df['end_cluster'])}
pcs = list()
eclus = list()
for re in res:
    pc = res[re]
    min_dist = 300
    eclu = None
    for ecp in ecs:
        dist = haversine_distance(re, ecp)
        if dist < min_dist:
            min_dist = dist
            eclu = ecs[ecp]
            
    pcs.append(pc)
    eclus.append(eclu)

Counter(np.array(pcs) == np.array(eclus))

Counter({False: 18, True: 4})

In [255]:
class KnownStartEstimator(FuzzyLocationMixin):
    def __init__(
            self,
            time_window = timedelta(hours=1),
            look_ahead = timedelta(hours=1),
            max_dist_nearest_cluster = 100,
            
    ):
        self.time_window = time_window
        self.dummydate = datetime.date(1970, 1, 1)
        self.look_ahead = look_ahead
        self.max_dist_nearest_cluster = max_dist_nearest_cluster
        

    def fit(self, X, y=None):
        self.data_ = [(date, start, lat, lon, end) for date, start, lat, lon, end in 
                      zip(X['start_date'], X['start_cluster'], X['start_lat'], X['start_lon'], X['end_cluster'])]
        
        look_ahead_data = list()
        idx_insert = list()
        if self.look_ahead:
            for i, tup in enumerate(self.data_):
                for j in range(i+1, len(self.data_)):
                    if self.data_[j][0] - tup[0] < self.look_ahead:
                        look_ahead_data.append([tup[0], tup[1], tup[2], tup[3], self.data_[j][4]])
                        idx_insert.append(i+1)
        
        idx_insert = sorted(idx_insert, reverse=True)
        for i, idx in enumerate(idx_insert):
            self.data_.insert(idx, look_ahead_data[i])
        
        self.data_ = pd.DataFrame(data=self.data_, columns=['start_date','start_cluster','start_lat','start_lon','end_cluster'])
        self.data_ = self.data_.set_index(pd.DatetimeIndex(self.data_['start_date'])).sort_index()
        
        return self

    def predict(self, traj):
        try:
            return self.predict_proba(traj).index[0]
        except:
            return None


        return res
    
    def predict_proba(self, traj):
        res_list = []
        length = len(self.data_)
        first_point = traj[0]
        
        lat = first_point.lat
        lon = first_point.lon
        nearest_cluster = self._get_nearest_cluster(lat, lon)
        if nearest_cluster == None:
            return None

        res = self.data_[self.data_['start_cluster'] == nearest_cluster]

        try:
            res = res['end_cluster'].value_counts() / length
        except:
            res = None

        return res


In [366]:
kse = KnownStartEstimator()
kse.fit(df[:81])

he = HeadingEstimator()
he.fit(df[:81])
res = dict()
for traj in test_trajs:
    
    if len(traj) < 100:
        continue
    
    print(kse.predict(traj[0:100]))
    print(he.predict(traj[0:100]))
    print(get_nearest_end_cluster(df, traj[-1].lat, traj[-1].lon))
    print()

1
12
1

3
15
3

1
16
1

3
24
3

1
16
1

3
6
3

1
16
1

3
6
3

3
6
3

1
3
1

3
6
3

1
16
1

3
24
26

None
6
26

None
22
3

None
3
3

1
2
1

None
10
1

3
6
3

1
3
1

3
15
3

1
3
19

None
10
19

None
13
3

1
16
1

3
15
3

1
13
9

3
3
1

3
15
3

1
3
1

3
1
3

1
3
1

3
6
20

None
6
1

3
6
27

3
24
3



In [239]:
ecs = {Point(lat, lon, None) : end for lat, lon, end in zip(df['end_lat'], df['end_lon'], df['end_cluster'])}
pcs = list()
eclus = list()
for re in res:
    pc = res[re]
    min_dist = 300
    eclu = None
    for ecp in ecs:
        dist = haversine_distance(re, ecp)
        if dist < min_dist:
            min_dist = dist
            eclu = ecs[ecp]
            
    pcs.append(pc)
    eclus.append(eclu)

Counter(np.array(pcs) == np.array(eclus))

Counter({False: 31, True: 5})

In [248]:
def get_nearest_end_cluster(df, lat, lon):
    def haversine_distance(p1_lat, p1_lon, p2_lat, p2_lon):
            lat_rad1 = radians(p1_lat)
            lon_rad1 = radians(p1_lon)
            lat_rad2 = radians(p2_lat)
            lon_rad2 = radians(p2_lon)
            return 2*R * asin(sqrt(sin((lat_rad2-lat_rad1)/2)**2 + cos(lat_rad1)*cos(lat_rad2)*(sin((lon_rad2-lon_rad1)/2)**2)))
    
    
    min_dist = 300
    cluster = None
    for index, row in df.iterrows():

        dist = haversine_distance(lat, lon, row['end_lat'], row['end_lon'])
        if dist < min_dist:
            cluster = row['end_cluster']
            min_dist = dist

    return cluster


In [362]:
class KnownStartDepartureTimeEstimator(FuzzyLocationMixin):
    def __init__(
            self,
            time_window = timedelta(hours=1),
            look_ahead = timedelta(hours=1),
            max_dist_nearest_cluster = 100,
            
    ):
        self.time_window = time_window
        self.dummydate = datetime.date(1970, 1, 1)
        self.look_ahead = look_ahead
        self.max_dist_nearest_cluster = max_dist_nearest_cluster
        

    def fit(self, X, y=None):
        self.data_ = [(date, start, lat, lon, end) for date, start, lat, lon, end in 
                      zip(X['start_date'], X['start_cluster'], X['start_lat'], X['start_lon'], X['end_cluster'])]
        
        look_ahead_data = list()
        idx_insert = list()
        if self.look_ahead:
            for i, tup in enumerate(self.data_):
                for j in range(i+1, len(self.data_)):
                    if self.data_[j][0] - tup[0] < self.look_ahead:
                        look_ahead_data.append([tup[0], tup[1], tup[2], tup[3], self.data_[j][4]])
                        idx_insert.append(i+1)
        
        idx_insert = sorted(idx_insert, reverse=True)
        for i, idx in enumerate(idx_insert):
            self.data_.insert(idx, look_ahead_data[i])
        
        self.data_ = pd.DataFrame(data=self.data_, columns=['start_date','start_cluster','start_lat','start_lon','end_cluster'])
        self.data_ = self.data_.set_index(pd.DatetimeIndex(self.data_['start_date'])).sort_index()
        
        return self

    def predict(self, traj):
        try:
            return self.predict_proba(traj).index[0]
        except:
            return None

    
    def predict_proba(self, traj):
        res_list = []
        length = len(self.data_)
        first_point = traj[0]
        
        lat = first_point.lat
        lon = first_point.lon
        nearest_cluster = self._get_nearest_cluster(lat, lon)
        if nearest_cluster == None:
            return None

        res = self.data_[self.data_['start_cluster'] == nearest_cluster]
        
        dummydatetime = datetime.datetime.combine(self.dummydate, datetime.time(first_point.datetime.hour, first_point.datetime.minute))
        lowerbound = dummydatetime - self.time_window
        upperbound = dummydatetime + self.time_window
        lowerbound = '{}:{}'.format(lowerbound.hour, lowerbound.minute)
        upperbound = '{}:{}'.format(upperbound.hour, upperbound.minute)
        res = res.between_time(lowerbound, upperbound)
        
        try:
            return res['end_cluster'].value_counts() / length
        except:
            return None



In [367]:
kse = KnownStartDepartureTimeEstimator()
kse.fit(df[:81])

ks = KnownStartEstimator()
ks.fit(df[:81])


res = dict()
for traj in test_trajs:
    
    if len(traj) < 100:
        continue
    
    print(kse.predict(traj[0:100]))
    print(ks.predict(traj[0:100]))
    print(get_nearest_end_cluster(df, traj[-1].lat, traj[-1].lon))
    print()

1
1
1

3
3
3

1
1
1

3
3
3

None
1
1

3
3
3

1
1
1

3
3
3

3
3
3

1
1
1

3
3
3

1
1
1

3
3
26

None
None
26

None
None
3

None
None
3

3
1
1

None
None
1

3
3
3

1
1
1

3
3
3

3
1
19

None
None
19

None
None
3

1
1
1

3
3
3

3
1
9

1
3
1

3
3
3

1
1
1

3
3
3

1
1
1

1
3
20

None
None
1

None
3
27

3
3
3

